In [1]:
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy
import os
import torch
import torch.nn as nn
import os
import torchvision as tv
from torchvision import transforms
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import time
import copy
from torch.optim import Adam
plt.ion()  

In [2]:
use_gpu = 0
if use_gpu:
    torch.cuda.set_device(CUDA_DEVICE)

count=0

In [3]:
char = 'A'
class_names = [chr(i) for i in range(0,256)]
print(class_names)
print(len(class_names))

['\x00', '\x01', '\x02', '\x03', '\x04', '\x05', '\x06', '\x07', '\x08', '\t', '\n', '\x0b', '\x0c', '\r', '\x0e', '\x0f', '\x10', '\x11', '\x12', '\x13', '\x14', '\x15', '\x16', '\x17', '\x18', '\x19', '\x1a', '\x1b', '\x1c', '\x1d', '\x1e', '\x1f', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '\x7f', '\x80', '\x81', '\x82', '\x83', '\x84', '\x85', '\x86', '\x87', '\x88', '\x89', '\x8a', '\x8b', '\x8c', '\x8d', '\x8e', '\x8f', '\x90', '\x91', '\x92', '\x93', '\x94', '\x95', '\x96', '\x97', '\x98', '\x99', '\x9a', '\x9b', '\x9c', '\x9d', '\x9e', '\x9f', '\xa0', '

In [4]:
#load data
data_folder = "RNN"
file = open("C:/Users/Gayatri Narang/Documents/682/autoproject/RNN/data2.txt", 'r')
data = file.read().replace('\n', ' ').upper()
data = ' '+data+' '

In [5]:
import string
printable = set(string.printable)
data = (list(filter(lambda x: x in printable, data)))
data = ("").join(data)
print(data)

 VACCINES ARE ONE OF THE GREAT SUCCESS STORIES OF MODERN MEDICINE. BECAUSE OF THEM WE ARE NO LONGER VULNERABLE TO SMALLPOX OR POLIO OR MEASLES. THE FLU VACCINE, HOWEVER, IS A DIFFERENT STORY. ITS EFFECTIVENESS VARIES FROM PATIENT TO PATIENT, FROM POPULATION TO POPULATION, AND FROM YEAR TO YEAR. IT NEEDS TO BE UPDATED EACH SEASON, AND EVEN IN A GOOD YEAR IS USUALLY NO MORE THAN 50% EFFECTIVE. WE MAY RELY ON IT TO AVOID CATCHING THE FLU, BUT ITS STORY DEMONSTRATES HOW FAR WE STILL ARE FROM A RELIABLE VACCINE. LOSE YOURSELF IN A GREAT STORY: SIGN UP FOR THE LONG READ EMAIL VACCINATION, THE PROCESS OF INFECTING A HEALTHY PERSON WITH A MICROBE TO PREVENT DISEASE, DATES BACK AT LEAST A THOUSAND YEARS. BUT THE START OF VACCINATION AS WE THINK OF IT TODAY IS GENERALLY CREDITED TO THE WORK OF EDWARD JENNER, A BRITISH PHYSICIAN BORN IN 1749. JENNER WAS A KEEN OBSERVER WITH A DEEP INTEREST IN THE NATURAL WORLD, AND FOUND TIME FOR BOTH SERIOUS STUDY AND ARTISTIC PLAY. HE INVESTIGATED EVERYTHING FR

In [6]:
#data = data.encode('ascii',errors='ignore')
print(data)

 VACCINES ARE ONE OF THE GREAT SUCCESS STORIES OF MODERN MEDICINE. BECAUSE OF THEM WE ARE NO LONGER VULNERABLE TO SMALLPOX OR POLIO OR MEASLES. THE FLU VACCINE, HOWEVER, IS A DIFFERENT STORY. ITS EFFECTIVENESS VARIES FROM PATIENT TO PATIENT, FROM POPULATION TO POPULATION, AND FROM YEAR TO YEAR. IT NEEDS TO BE UPDATED EACH SEASON, AND EVEN IN A GOOD YEAR IS USUALLY NO MORE THAN 50% EFFECTIVE. WE MAY RELY ON IT TO AVOID CATCHING THE FLU, BUT ITS STORY DEMONSTRATES HOW FAR WE STILL ARE FROM A RELIABLE VACCINE. LOSE YOURSELF IN A GREAT STORY: SIGN UP FOR THE LONG READ EMAIL VACCINATION, THE PROCESS OF INFECTING A HEALTHY PERSON WITH A MICROBE TO PREVENT DISEASE, DATES BACK AT LEAST A THOUSAND YEARS. BUT THE START OF VACCINATION AS WE THINK OF IT TODAY IS GENERALLY CREDITED TO THE WORK OF EDWARD JENNER, A BRITISH PHYSICIAN BORN IN 1749. JENNER WAS A KEEN OBSERVER WITH A DEEP INTEREST IN THE NATURAL WORLD, AND FOUND TIME FOR BOTH SERIOUS STUDY AND ARTISTIC PLAY. HE INVESTIGATED EVERYTHING FR

In [7]:
def to_categorical(y, num_classes):
    """ 1-hot encodes a tensor """
    return ( torch.from_numpy(np.eye(num_classes)[y]))

In [8]:
vocab_ind = {}
ind_vocab = {}
for (index, letter) in enumerate(class_names):
    vocab_ind[letter] = index
    ind_vocab[index] = letter
# vocab_ind[" "] = index+1
# ind_vocab[index+1] = " "

In [9]:
print(vocab_ind, ind_vocab)

{'\x00': 0, '\x01': 1, '\x02': 2, '\x03': 3, '\x04': 4, '\x05': 5, '\x06': 6, '\x07': 7, '\x08': 8, '\t': 9, '\n': 10, '\x0b': 11, '\x0c': 12, '\r': 13, '\x0e': 14, '\x0f': 15, '\x10': 16, '\x11': 17, '\x12': 18, '\x13': 19, '\x14': 20, '\x15': 21, '\x16': 22, '\x17': 23, '\x18': 24, '\x19': 25, '\x1a': 26, '\x1b': 27, '\x1c': 28, '\x1d': 29, '\x1e': 30, '\x1f': 31, ' ': 32, '!': 33, '"': 34, '#': 35, '$': 36, '%': 37, '&': 38, "'": 39, '(': 40, ')': 41, '*': 42, '+': 43, ',': 44, '-': 45, '.': 46, '/': 47, '0': 48, '1': 49, '2': 50, '3': 51, '4': 52, '5': 53, '6': 54, '7': 55, '8': 56, '9': 57, ':': 58, ';': 59, '<': 60, '=': 61, '>': 62, '?': 63, '@': 64, 'A': 65, 'B': 66, 'C': 67, 'D': 68, 'E': 69, 'F': 70, 'G': 71, 'H': 72, 'I': 73, 'J': 74, 'K': 75, 'L': 76, 'M': 77, 'N': 78, 'O': 79, 'P': 80, 'Q': 81, 'R': 82, 'S': 83, 'T': 84, 'U': 85, 'V': 86, 'W': 87, 'X': 88, 'Y': 89, 'Z': 90, '[': 91, '\\': 92, ']': 93, '^': 94, '_': 95, '`': 96, 'a': 97, 'b': 98, 'c': 99, 'd': 100, 'e': 101

In [10]:
def embedd(word):
    x = torch.empty(6)
    c = 0
    word = word.upper() 
    word = word[:6]
    for i in word:
        x[c] = vocab_ind[i]
        c += 1
    x = x.type(torch.LongTensor)    
    return x    

In [11]:
import torch
import torch.nn as nn
from torch.autograd import Variable

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        self.encoder = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
    
    def forward(self, input, hidden):
        input = self.encoder(input.view(1, -1))
        output, hidden = self.gru(input.view(1, 1, -1), hidden)
        output = self.decoder(output.view(1, -1))
        return output, hidden

    def init_hidden(self):
        return Variable(torch.zeros(self.n_layers, 1, self.hidden_size))
        #return Variable(torch.randn(self.n_layers, 1, self.hidden_size))

In [12]:
def trainRNN(inp, target):
    hidden = decoder.init_hidden()
    decoder.zero_grad()
    loss = 0
    chunk_len = 6
    for c in range(chunk_len):
        output, hidden = decoder(inp[c], hidden)
        value, index = torch.max(output.data, 1)
        loss += criterion(output, target[c].view((1)))
    #print(loss)
    loss.backward()
    decoder_optimizer.step()

    return loss.data[0] / chunk_len

In [13]:
#model.eval()

In [14]:
n_epochs = 100
print_every = 100
plot_every = 100
hidden_size = 100
n_layers = 1
lr = 0.001
n_characters = 256

decoder = RNN(n_characters, hidden_size, n_characters, n_layers)
for param in decoder.parameters():
    param.requires_grad = True
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

start = time.time()
all_losses = []
loss_avg = 0

for epoch in range(1, n_epochs + 1):
    print(epoch)
    loss_avg = 0
    for  i in range(0,len(data),6):
        emb_data = embedd(data[i:i+6])
        emb_out = embedd(data[i+1:i+7])
        cap_in = emb_data
        cap_out = emb_out
        #print(cap_in)
        #print(cap_out)

        loss = trainRNN(cap_in, cap_out)       
        loss_avg += loss

#         if epoch % print_every == 0:
#             print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / n_epochs * 100, loss))
#             print(evaluate('Wh', 100), '\n')

#         if epoch % plot_every == 0:
#             all_losses.append(loss_avg / plot_every)
#             loss_avg = 0
    print(loss_avg)

1


C:\anaconda3\envs\cs682\lib\site-packages\ipykernel_launcher.py:14: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  


tensor(3349.6519)
2
tensor(2821.4497)
3
tensor(2645.9749)
4
tensor(2513.1350)
5
tensor(2403.3838)
6
tensor(2306.1963)
7
tensor(2218.0405)
8
tensor(2136.9102)
9
tensor(2060.4307)
10
tensor(1987.2524)
11
tensor(1922.0671)
12
tensor(1860.8345)
13
tensor(1802.6156)
14
tensor(1753.3983)
15
tensor(1703.7166)
16
tensor(1657.2273)
17
tensor(1614.9011)
18
tensor(1582.4821)
19
tensor(1543.2255)
20
tensor(1514.6936)
21
tensor(1480.6678)
22
tensor(1461.7809)
23
tensor(1433.4196)
24
tensor(1403.2604)
25
tensor(1387.5129)
26
tensor(1361.5361)
27
tensor(1351.8588)
28
tensor(1325.8966)
29
tensor(1310.0259)
30
tensor(1288.7798)
31
tensor(1282.2983)
32
tensor(1271.1376)
33
tensor(1255.4426)
34
tensor(1246.3207)
35
tensor(1235.8042)
36
tensor(1221.3060)
37
tensor(1219.2736)
38
tensor(1215.2432)
39
tensor(1212.6450)
40
tensor(1199.9084)
41
tensor(1192.4567)
42
tensor(1201.8262)
43
tensor(1186.5038)
44
tensor(1185.7316)
45
tensor(1176.5436)
46
tensor(1172.4677)
47
tensor(1168.7256)
48
tensor(1168.7227)
49


In [15]:
def evaluate(inp):
    hidden = decoder.init_hidden()
    chunk_len = 6
    predicted = torch.zeros([chunk_len])
    for c in range(chunk_len):
        output, hidden = decoder(inp[c], hidden)
        value, index = torch.max(output.data, 1)
        predicted[c] = index
    return predicted

In [18]:
for param in decoder.parameters():
    param.requires_grad = False
test_data = ' vaccination '
for  i in range(0,len(test_data),6):
        #outputs = m(images)
        #_, prediction = torch.max(outputs.data, 1)
        #print("CNN")
        #print(prediction)
        #space = torch.tensor([28])
        #space.type(torch.LongTensor)
        #cap_in = torch.cat((space,prediction)) 
        cap_in = test_data[i:i+6]
        emb_cap_in = embedd(cap_in)
        #print(emb_cap_in)
        p = evaluate(emb_cap_in)
        #print("RNN")
        for i in range(6):
            print(ind_vocab[int(p[i])].lower(), end = "")

taccindcion,t xxxx